In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle
from scipy.sparse import csr_matrix
from tqdm import tqdm
import gc
import psutil



In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d undefinenull/million-song-dataset-spotify-lastfm

Dataset URL: https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)


In [4]:
!unzip million-song-dataset-spotify-lastfm.zip

Archive:  million-song-dataset-spotify-lastfm.zip
  inflating: MP3-Example/Blues/Blues-TRACOHF128F1498509.mp3  
  inflating: MP3-Example/Blues/Blues-TRADWSG128F4259317.mp3  
  inflating: MP3-Example/Blues/Blues-TRAELUY128F93147BA.mp3  
  inflating: MP3-Example/Blues/Blues-TRAHVWB128F9308FB0.mp3  
  inflating: MP3-Example/Blues/Blues-TRAJGIO128F92E84BC.mp3  
  inflating: MP3-Example/Blues/Blues-TRAOPNA128F427E83F.mp3  
  inflating: MP3-Example/Blues/Blues-TRAQGFP128E078FAB6.mp3  
  inflating: MP3-Example/Blues/Blues-TRARJEK128F930B3AA.mp3  
  inflating: MP3-Example/Blues/Blues-TRATLTV128F92FC979.mp3  
  inflating: MP3-Example/Blues/Blues-TRAUJPR128F92CA3AF.mp3  
  inflating: MP3-Example/Blues/Blues-TRAUYOD128F42647FE.mp3  
  inflating: MP3-Example/Blues/Blues-TRAYHQV128F4266D63.mp3  
  inflating: MP3-Example/Blues/Blues-TRBVOHY128F92E6A11.mp3  
  inflating: MP3-Example/Blues/Blues-TRCPUIL128F931ACAF.mp3  
  inflating: MP3-Example/Blues/Blues-TRCQEDW128E078F467.mp3  
  inflating: MP3-Exa

In [5]:
music_hist=pd.read_csv('/content/Music Info.csv')

In [5]:
music_hist.shape

(50683, 21)

In [8]:
music_hist.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [6]:
user_hist=pd.read_csv('/content/User Listening History.csv')

In [7]:
user_hist.shape

(9711301, 3)

In [10]:
user_hist.head()

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [7]:
# Merge the dataframes on the 'track_id' column
merged_df = pd.merge(user_hist, music_hist, on='track_id', how='left')

In [9]:
merged_df.shape

(9711301, 23)

In [8]:
# Randomly sample 100,000 rows from the merged dataframe
merged_sample = merged_df.sample(n=100000, random_state=42)

# Check the shape to be sure
print(merged_sample.shape)

(100000, 23)


In [14]:
merged_sample.head()

,track_id,user_id,playcount,name,artist,spotify_preview_url,spotify_id,tags,genre,year,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2508064,TRNTNZL128F4259856,449bfa756a8e71f501aed4a84250aa1237f0040a,6,Warriors Of Time,Black Tide,https://p.scdn.co/mp3-preview/422c550a1fcd2708...,0nrSYmFi8NKxnBClCUF1xZ,"metal, heavy_metal, thrash_metal",Metal,2008,...,5,-3.724,1,0.1760,0.000045,0.019600,0.132,0.203,117.516,4
4671124,TRNBMJP128F93433D8,b218f76e5053c281ba233cf43845a285bfc3049f,1,Kilometros,Sin Bandera,https://p.scdn.co/mp3-preview/439f464362d93fec...,0AcFsf9FGjL1Al2RBQ1Crq,classical,NaN,2001,...,9,-8.170,1,0.0283,0.303000,0.000000,0.118,0.277,129.975,4
3094899,TRDYGUV12903CDAF1E,533cbbc0be49822ce40f1d7fce2dc5db2dd377e1,1,A Mutiny,Red Sparowes,https://p.scdn.co/mp3-preview/e6570c5e1fd185ff...,4i39VKIzzOgVQS1OPoXvFG,"experimental, instrumental, post_rock",NaN,2010,...,2,-7.984,0,0.0320,0.200000,0.878000,0.108,0.130,135.448,4
134535,TRUSRFO128F147FC82,9c0e51864b053caeb212fefd0843de3af6a19283,1,That's Just the Way It Is,Phil Collins,https://p.scdn.co/mp3-preview/5e513b57761cafc7...,1mQat4bUvjF4RycotIOcF7,"rock, pop, 80s, pop_rock",Rock,1989,...,5,-19.871,1,0.0251,0.298000,0.000159,0.379,0.333,96.004,4
125426,TRITUFI128F147B77A,57255430f4d62c72429c1b3808b64ac1bca08144,2,Another Town,Regina Spektor,https://p.scdn.co/mp3-preview/5a367d0bc1ac2fa8...,2O4uO8jHAULYH5gMIy9eVD,"female_vocalists, piano, russian",NaN,2006,...,0,-9.784,1,0.0311,0.954000,0.634000,0.124,0.238,175.027,4


In [13]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711301 entries, 0 to 9711300
Data columns (total 23 columns):
 #   Column               Dtype  
---  ------               -----  
 0   track_id             object 
 1   user_id              object 
 2   playcount            int64  
 3   name                 object 
 4   artist               object 
 5   spotify_preview_url  object 
 6   spotify_id           object 
 7   tags                 object 
 8   genre                object 
 9   year                 int64  
 10  duration_ms          int64  
 11  danceability         float64
 12  energy               float64
 13  key                  int64  
 14  loudness             float64
 15  mode                 int64  
 16  speechiness          float64
 17  acousticness         float64
 18  instrumentalness     float64
 19  liveness             float64
 20  valence              float64
 21  tempo                float64
 22  time_signature       int64  
dtypes: float64(9), int64(6), object(

In [9]:
spotify_merge =merged_sample[['user_id','name','playcount','artist','tags','year','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

In [15]:
spotify_merge .head()

,user_id,name,playcount,artist,tags,year,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Nothing From Nothing,1,Billy Preston,"soul, funk, piano, 70s, oldies",2010,0.834,9,-5.206,0,0.0276,0.58500,0.055000,0.3310,0.939,102.792
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Paper Gangsta,1,Lady Gaga,"electronic, pop, female_vocalists, dance, pian...",2012,0.530,10,-9.344,0,0.0579,0.80400,0.782000,0.1050,0.800,100.002
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Soy,1,Gipsy Kings,guitar,2003,0.592,5,-11.647,1,0.0673,0.69200,0.000002,0.1100,0.851,104.518
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Stacked Actors,1,Foo Fighters,"rock, alternative, alternative_rock, hard_rock...",1999,0.934,7,-5.145,1,0.0500,0.00203,0.001020,0.2340,0.670,134.898
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Heaven's Gonna Burn Your Eyes,1,Thievery Corporation,"electronic, female_vocalists, ambient, chillou...",2002,0.337,9,-9.812,1,0.0286,0.63500,0.521000,0.0637,0.323,152.411


In [10]:
spotify_merge.drop_duplicates(subset=['name'], inplace=True)

<ipython-input-10-6c14e82f3713>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_merge.drop_duplicates(subset=['name'], inplace=True)


In [11]:
spotify_merge = spotify_merge.dropna().reset_index(drop=True)

In [12]:
print(len(spotify_merge ))

15088


In [13]:
spotify_merge['name'].duplicated().sum()

np.int64(0)

In [14]:
spotify_merge ['playcount'].value_counts()

,count
playcount,
1,9495
2,2338
3,942
5,568
4,540
6,273
7,175
8,116
10,98


In [15]:
# 🗂 Load your dataset
df = spotify_merge   # replace with your file path

In [16]:
df.sample(5)

,user_id,name,playcount,artist,tags,year,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
3114,5f2087e8f0171a2b5ed6258e158c75b921411e79,My Little Corner of the World,1,Yo La Tengo,"alternative, indie, soundtrack, 90s, acoustic,...",1997,0.335,3,-14.640,1,0.0287,0.864000,0.916000,0.0976,0.598,130.484
2192,ec61a3b54cc0a0004ee86885a025414a2dc45143,Arson,1,Amon Amarth,"death_metal, melodic_death_metal",2004,0.931,3,-3.356,0,0.0447,0.000005,0.766000,0.1260,0.233,133.744
4934,10bebc039421958b5dcc7d4b36b7a961221ecefc,The Last Something That Meant Anything,2,Mayday Parade,"alternative, emo",2006,0.749,7,-5.355,1,0.0262,0.005530,0.000612,0.1440,0.298,74.991
5447,5c8c3971adc14bf3b7d4ede3ab1addb0a3eaab64,Millions,3,Starfucker,"electronic, alternative, alternative_rock, ind...",2011,0.862,6,-6.651,1,0.0425,0.000478,0.019400,0.1690,0.530,125.058
3952,0386102f4de2ac288725906f754defb68e01f323,I Can't Breathe,2,Krypteria,"power_metal, gothic_metal, symphonic_metal",2007,0.927,1,-4.832,0,0.0483,0.082700,0.000009,0.0645,0.547,94.830


In [17]:
# 🎯 Create binary label: repeated or not
df['repeated_within_month'] = df['playcount'].apply(lambda x: 1 if x >= 2 else 0)

In [18]:
df['tags'] = df['tags'].astype(str).str.lower()

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Cell 3 - TF-IDF on 'tags'
tfidf = TfidfVectorizer(stop_words='english')
tag_matrix = tfidf.fit_transform(df['tags'])

# Compute cosine similarity between songs based on tags
tag_similarity = cosine_similarity(tag_matrix, tag_matrix)


In [20]:
# Cell 4 - Collaborative filtering matrix
user_song_matrix = df.pivot_table(index='user_id', columns='name', values='playcount', fill_value=0)

# Sparse matrix format
song_user_mat_sparse = csr_matrix(user_song_matrix.T.values)

# Cosine similarity between songs
collab_similarity = cosine_similarity(song_user_mat_sparse)
song_names = user_song_matrix.columns.tolist()


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
import pandas as pd

def create_features(df):
    # Numerical features
    feature_cols = ['energy', 'key', 'loudness', 'mode',
                    'speechiness', 'acousticness', 'instrumentalness',
                    'liveness', 'valence', 'tempo']

    # Text features
    df['tags'] = df['tags'].str.replace(', ', ',')  # Normalize tags
    df['artist_tags'] = df['artist'] + ' ' + df['tags']  # Combined feature

    # Interaction features
    df['energy_valence'] = df['energy'] * df['valence']
    df['acoustic_speech'] = df['acousticness'] * df['speechiness']

    return df

# Load and preprocess data (assuming df is your DataFrame)
# Replace this with your actual data loading code
# df = pd.read_csv('your_data.csv')
df = create_features(df)  # CREATE FEATURES FIRST

# THEN define feature sets
feature_cols = ['energy', 'key', 'loudness', 'mode', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness', 'valence',
                'tempo', 'energy_valence', 'acoustic_speech']

text_cols = ['artist', 'tags', 'artist_tags']

# Encode user_id
le_user = LabelEncoder()
df['user_id'] = le_user.fit_transform(df['user_id'])

# Fill missing values
df[feature_cols] = df[feature_cols].fillna(df[feature_cols].mean())
df[text_cols] = df[text_cols].fillna('unknown')  # Fill missing text with 'unknown'

# Rest of your code remains the same...

# Preprocessing for text data
text_transformer = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english'))
])

# Preprocessing for numerical data
numeric_transformer = Pipeline([
    ('scaler', MinMaxScaler())
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, feature_cols),
        ('artist', text_transformer, 'artist'),
        ('tags', text_transformer, 'tags'),
        ('artist_tags', text_transformer, 'artist_tags')
    ])

# Split data
X = df[feature_cols + text_cols + ['user_id']]
y = df['repeated_within_month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline with preprocessing and XGBoost
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),  # Handle class imbalance

    ))
])

# Fit the model
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.62      0.63      1875
           1       0.42      0.46      0.44      1143

    accuracy                           0.56      3018
   macro avg       0.54      0.54      0.54      3018
weighted avg       0.56      0.56      0.56      3018



In [25]:
# Hybrid recommendation function
def hybrid_recommend(user_id, song_name, top_n=10):
    if song_name not in df['name'].values:
        return f"Song '{song_name}' not found."

    # Content-based (tags)
    song_idx = df[df['name'] == song_name].index[0]
    tag_scores = list(enumerate(tag_similarity[song_idx]))

    # Collaborative filtering
    if song_name in song_names:
        idx = song_names.index(song_name)
        collab_scores_raw = list(enumerate(collab_similarity[idx]))
        # Map collaborative scores to df indices
        collab_scores = [(i, 0) for i in range(len(df))]  # Default score of 0
        song_name_to_idx = {name: i for i, name in enumerate(song_names)}
        for df_idx, name in enumerate(df['name']):
            if name in song_name_to_idx:
                collab_idx = song_name_to_idx[name]
                collab_scores[df_idx] = (df_idx, collab_scores_raw[collab_idx][1])
    else:
        collab_scores = [(i, 0) for i in range(len(df))]  # Default score of 0

    # Repeated play prediction
    song_features = df[feature_cols + text_cols + ['user_id']].copy()

    # Ensure user_id is encoded consistently
    if user_id not in le_user.classes_:
        song_features['user_id'] = 0  # Default user_id for unknown users
    else:
        song_features['user_id'] = le_user.transform([user_id])[0]

    # Apply the same preprocessing as in training
    song_features_preprocessed = model.named_steps['preprocessor'].transform(song_features)

    # Predict repeat probability
    repeat_proba = model.named_steps['classifier'].predict_proba(song_features_preprocessed)[:, 1]

    # Combine all scores
    final_scores = []
    for i in range(len(df)):
        score = (0.4 * tag_scores[i][1]) + \
                (0.3 * collab_scores[i][1]) + \
                (0.3 * repeat_proba[i])
        final_scores.append((i, score))

    # Sort and return top N
    final_scores = sorted(final_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in final_scores[:top_n]]
    return df.iloc[top_indices][['name', 'artist', 'tags']]

# Test the recommendation
recommendations = hybrid_recommend(user_id='5c8c3971adc14bf3b7d4ede3ab1addb0a3eaab64', song_name='Millions', top_n=10)
print(recommendations)

                             name           artist  \
5447                     Millions       Starfucker   
11745              Flash Delirium             MGMT   
3636   4th Dimensional Transition             MGMT   
1768                 It's Working             MGMT   
3328            Someone's Missing             MGMT   
6328                 Show You How      The Killers   
1607                  Uncle Jonny      The Killers   
1678    Neighborhood #1 (Tunnels)      Arcade Fire   
544    Of Moons, Birds & Monsters             MGMT   
10551          Ribbon On a Branch  Younger Brother   

                                                    tags  
5447   electronic,alternative,alternative_rock,indie_...  
11745  rock,electronic,alternative,indie,pop,alternat...  
3636   rock,electronic,alternative,indie,indie_rock,p...  
1768   electronic,alternative,indie,pop,alternative_r...  
3328   rock,electronic,indie,pop,alternative_rock,ind...  
6328   rock,alternative_rock,indie_rock,psychedelic